In [1]:
import torch
import numpy as np

def generate_data(seq_len: int, num_samples: int):
    X = []
    y = []
    for i in range(num_samples):
        start = np.random.rand() * 2 * np.pi
        sequence = np.sin(np.linspace(start, start + seq_len * 0.1, seq_len))
        X.append(sequence)
        y.append(np.sin(start + seq_len * 0.1))
    # end for
    return np.array(X), np.array(y)
# end def

seq_len = 20 
num_samples = 500 
input_size = 1
hidden_size = 50
output_size = 1
num_layers = 2
epochs = 200

X, y = generate_data(seq_len, num_samples)

X_train = torch.tensor(X[:450], dtype=torch.float32).unsqueeze(-1)
y_train = torch.tensor(y[:450], dtype=torch.float32).unsqueeze(-1)

X_test = torch.tensor(X[450:], dtype=torch.float32).unsqueeze(-1)
y_test = torch.tensor(y[450:], dtype=torch.float32).unsqueeze(-1)

In [2]:
from Models.ModelWrapper import ModelWrapper
from torch.optim import RMSprop, AdamW, Adam, NAdam

args = {
    'hidden_size' : 25,
    'num_layers' : 3,
    'seq_len' : 20,
    'optimazer' : 'rmsprop'
}

model = ModelWrapper(type = 'lstm', **args)

model.fit(X_train, y_train, epochs = 200)
model.score(X_test, y_test)

0.0006569924880750477

In [3]:
args = {
    'hidden_size' : 25,
    'num_layers' : 3,
    'seq_len' : 20,
    'optimazer' : 'nadam'
}

model = ModelWrapper(type = 'gru', **args)

model.fit(X_train, y_train, epochs = 200)
model.score(X_test, y_test)

0.00021163575001992285